## Python Notebook to convert exam timetabling xml files to .exam 

### File paths

In [16]:
input_file_path = 'xml\\pu-exam-fal08.xml'
output_file_path = 'xml\\exam-xml-conversion.exam'

### Import

In [17]:
from lxml import etree
from itertools import combinations

### XML file

In [18]:
# XML tree and root
tree = etree.parse(input_file_path)
root = tree.getroot()

### Period extraction

In [19]:
periods = root.find("periods").findall("period")

formatted_periods = []

for period in periods:
    id = period.get("id")               # Extracting id of period (e.g., 'id': '10')
    day = period.get("day")             # Extracting day and date of period (e.g., 'day': 'Mon 12/15')
    time = period.get("time")           # Extracting time of period (e.g., 'time': '3:20p - 5:20p') 
    length = period.get("length")       # Extracting length of period (e.g., 'length': '120')
    penalty = period.get("penalty")     # Extracting penalty of period (e.g., 'penalty': '0')

    # Splitting and converting date
    _, date_part = day.split()  # Extracting date part (e.g., '12/15')
    month, day = date_part.split("/")  # Splitting day and month
    formatted_date = f"{day}:{month}:2025"  # Converting to "day:month:2025"

    # Extracting the starting time
    start_time = time.split(" - ")[0]
    
    # Converting AM/PM to 24-hour format
    if start_time[-1] == 'a':  # Morning time
        hour, minute = start_time[:-1].split(":")
        hour = hour.zfill(2)  # Ensuring two-digit format (e.g., "08")
    else:  # Afternoon (PM)
        hour, minute = start_time[:-1].split(":")
        hour = str(int(hour) + 12) if hour != "12" else "12"

    # Formatting time (e.g., "08:00:00")
    formatted_time = f"{hour}:{minute}:00"
    
    # Combining all
    formatted_string = f"{formatted_date}, {formatted_time}, {length}, {penalty}"
    formatted_periods.append(formatted_string)

# Creation of starting string for Periods element
period_starting_string = f"[Periods:{id}]"

### Room

In [20]:
rooms = root.find("rooms").findall("room")

formatted_rooms = []

for room in rooms:  
    id = int(room.get("id"))      # Exctracting id of room (e.g., 'id': '10')
    size = int(room.get("size"))  # Extracting size of room (e.g., 'size': '10')
    
    # Checking for existence of <period> elements  
    if room.findall("period"):
        # Extracting all penalties from <period> elements
        penalties = [int(period.get("penalty", 0)) for period in room.findall("period")]
        
        # Checking if all penalties are the same
        penalty_value = penalties[0] if penalties and all(p == penalties[0] for p in penalties) else 0
    else: 
        penalty_value = 0
    
    # Combining all
    formatted_string = f"{size}, {penalty_value}"
    formatted_rooms.append(formatted_string)

# Creation of starting string for Rooms element
room_starting_string = f"[Rooms:{id}]"

### Exams

In [21]:
exams = root.find("exams").findall("exam")

# Dictionary to store length of exams
exam_lengths = {}

for exam in exams:
    id = int (exam.get("id"))           # Extracting id of exam (e.g., 'id': '10')
    length = int(exam.get("length"))    # Extracting length of exam (e.g., 'length': '120')

    exam_lengths[id] = length

students = root.find("students").findall("student")

# Dictionary to store students for each exam
exam_students = {}

for student in students:
    student_id = int(student.get("id"))                                         # Extracting id of student (e.g., 'id': '10')
    exams_list = [int(exam.get("id")) for exam in student.findall("exam")]      # Extracting exams of sudent (e.g., 'exam': '[2173, 302, 1546, 493] ')

    for exam_id in exams_list:
        if exam_id not in exam_students:                # Checking if exam is in dictionary
            exam_students[exam_id] = []                 # Initializing if not
        exam_students[exam_id].append(student_id)       # Appending student to exam

# Creation of starting string for Exams element
exam_starting_string = f"[Exams:{id}]"

### Constraints


AFTER            -- precedence 2 A 1 B

EXAM_COINCIDENCE -- same-period

EXCLUSION        -- different-period do for each exams

EXCLUSIVE        -- none 

In [22]:
# After
after_constraints = root.find("constraints").findall("precedence")

after_constraint_list = []

for after_constraint in after_constraints:
    exam_ids = [int(exam.get("id")) for exam in after_constraint.findall("exam")]   # Extracting ids of exams (e.g., 'id': '10') 

    if len(exam_ids) == 2:  # Ensuring there are exactly two exams
        after_constraint_list.append(f"{exam_ids[1]}, AFTER, {exam_ids[0]}")

# Coincidence
coincidence_constraints = root.find("constraints").findall("same-period")

coincidence_constraint_list = []

for coincidence_constraint in coincidence_constraints:
    exam_ids = [int(exam.get("id")) for exam in coincidence_constraint.findall("exam")]   # Extracting ids of exams (e.g., 'id': '10')

    for i in range(1, len(exam_ids)):   # Creating pairs (first exam with each of the others)
        coincidence_constraint_list.append(f"{exam_ids[0]}, EXAM_COINCIDENCE, {exam_ids[i]}")

# Exclusion
exclusion_constraints = root.find("constraints").findall("different-period")

exclusion_constraint_list = []

for exclusion_constraint in exclusion_constraints:
    exam_ids = [int(exam.get("id")) for exam in exclusion_constraint.findall("exam")]   # Extracting ids of exams (e.g., 'id': '10')

    for exam1, exam2 in combinations(exam_ids, 2):
        exclusion_constraint_list.append(f"{exam1}, EXCLUSION, {exam2}")

# Creation of starting string for PeriodHardConstraints element
period_hard_constraints_starting_string = "[PeriodHardConstraints]"

# Creation of string for RoomHardConstraints element
room_hard_constraints_string = "[RoomHardConstraints]"

### InstitutionalWeightings

TWOINAROW:  backToBackConflictWeight

TWOINADAY:  moreThanTwoADayWeight / 10

PERIODSPREAD: 5 do to no value being presented in the file

NONMIXEDDURATIONS: 10 do to no value being presented in the file

FRONTLOAD: 100,30,5  do to no value being presented in the file

In [23]:
parameters = root.find("parameters").findall("property")

parameter_list = []

for parameter in parameters:
    if parameter.get("name") == "backToBackConflictWeight":
        parameter_list.append(f"TWOINAROW, {int(float(parameter.get("value")))}")       # Extracting value of parameter (e.g., 'value': '10')
    if parameter.get("name") == "moreThanTwoADayWeight":
        parameter_list.append(f"TWOINADAY, {int(float(parameter.get("value")) / 10)}")  # Extracting value of parameter (e.g., 'value': '10') then dividing by 10

# Creation of string for InstitutionalWeightings element
parameter_starting_string = "[InstitutionalWeightings]"

parameter_list.append("PERIODSPREAD, 5")
parameter_list.append("NONMIXEDDURATIONS,10")
parameter_list.append("FRONTLOAD,100,30,5")

### File creation

In [24]:
with open(output_file_path, 'w') as file:
        file.write(exam_starting_string)
        file.write("\n")
        for exam_id in exam_lengths:
                students_str = ", ".join(map(str, exam_students.get(exam_id, [])))
                file.write(f"{exam_lengths[exam_id]}, {students_str}")
                file.write("\n")
        file.write(period_starting_string)
        file.write("\n")
        for fp in formatted_periods:
                file.write(fp)
                file.write("\n")
        file.write(room_starting_string)
        file.write("\n")
        for fr in formatted_rooms:
                file.write(fr)
                file.write("\n")
        file.write(period_hard_constraints_starting_string)
        file.write("\n")
        for acl in after_constraint_list:
                file.write(acl)
                file.write("\n")
        for ccl in coincidence_constraint_list:
                file.write(ccl)
                file.write("\n")
        for ecl in exclusion_constraint_list:
                file.write(ecl)
                file.write("\n")
        file.write(room_hard_constraints_string)
        file.write("\n")
        file.write(parameter_starting_string)
        file.write("\n")
        for pl in parameter_list:
                file.write(pl)
                file.write("\n")
        